In [1]:
#STIX simulator

In [2]:
import sys
sys.path.append('.')
import glob
import numpy as np
import math
from core.grid_parameters import NOMINAL as ngd


import matplotlib.pyplot as plt


from ipywidgets import interact, interact_manual, widgets


from IPython.display import clear_output

from core.stix_imager import StixImager
from core import stix_grid as sgd


%matplotlib notebook

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
def gaus2d( sun_x=0, sun_y=0, x_radius=30, y_radius=30, sigma=10, rho=0, steps=40, xmin=None, 
           xmax=None,  ymin=None, ymax=None):
    if xmin is None or ymin is None or xmax is None or ymax is None:
        xmin=sun_x-x_radius
        xmax=sun_x+x_radius
        ymin=sun_y-y_radius
        ymax=sun_y+y_radius
    xx,yy=np.meshgrid(np.linspace(xmin,xmax,steps), np.linspace(ymin,ymax,steps))
    u=(xx-sun_x)/sigma
    v=(yy-sun_y)/sigma
    c=1.-rho*rho
    z=u*u-2*rho*u*v+v*v
            # Calculating Gaussian array
    zz = np.exp(-( z/  2.0 * c ) )
    return xx,yy,zz
#x,y,z=gaus2d()

#plt.pcolormesh(x, y, z)

In [9]:
import pickle
steps=20
#detectors=[i for i in range(0,32) for i not in [8,9]]
detectors=[20,21]
xx=np.linspace(400,800,steps)
yy=np.linspace(400,800,steps)  

def create_pattern_lut():
    #res=20
    results=[]
    for det in detectors:
        im=StixImager(det)
        nd=len(detectors)

        print('Detector : ', det)
   

        for i in range(steps):
            for j in range(steps):
                clear_output(wait=True)
                x=xx[i]
                y=yy[j]
                print(f'completed: {100*(i*steps+j)/(steps*steps):.1f}%')
          
                res=im.create(x, y, 0,0,0,0, compute_pattern=True)
                results.append(res['pattern'])
    inv_resp=np.linalg.inv(results)
    np.savetxt('resp.csv', np.array(results), delimiter=",")
    np.savetxt('inv_resp.csv', np.array(results), delimiter=",")
    return np.array(results)
            


create_pattern_lut()

completed: 99.8%


LinAlgError: Last 2 dimensions of the array must be square

# Gaussian  shape source simulator

In [6]:



plt.close('all')
def simulate_gaus(det_idx, sun_x, sun_y, x_radius=30, y_radius=30,sigma=10,rho=0, steps=40
            ):
 
    
    im=StixImager(det_idx)
    
    
    xx,yy,zz=gaus2d(sun_x, sun_y, x_radius, y_radius, sigma, rho, steps)
    x1d=xx.reshape(-1)
    y1d=yy.reshape(-1)
    z1d=zz.reshape(-1)
    pattern=np.zeros(12)
    
    ns=x1d.size
    for i in range(ns):
        clear_output(wait=True)
        print(f'completed: {100.*i/ns:.1f}%')
        x=x1d[i]
        y=y1d[i]
        if z1d[i]>0:
            #print(x,y)
            res=im.create(x, y, 0,0,0,0, compute_pattern=True)
            pattern+=res['pattern']*z1d[i]
    res=im.create(sun_x, sun_y, 0,0,0,0, compute_pattern=True)
    fig, ((ax, ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize=(8, 8))
    ax.pcolormesh(xx, yy, zz)
    im.plot(fig, ax2, ax3)
    pattern=pattern/np.sum(z1d)
    ax3.plot(range(12),pattern, label='gaus')
    ax3.set_title(f'sun_x={sun_x} sun_y={sun_y} x_r={x_radius}, y_r={y_radius}, sigma={sigma} ')
    ax3.set_xlabel('Pixel')
    ax3.set_ylabel('Open area (mm)')
    ax3.legend()
    ax4.plot(range(12),pattern, label='gaus')
    ax4.set_title(f'sun_x={sun_x} sun_y={sun_y} x_r={x_radius}, y_r={y_radius}, sigma={sigma} ')
    ax4.set_xlabel('Pixel')
    ax4.set_ylabel('Open area (mm)')
    ax4.legend()
    plt.savefig(f'sun_x_{sun_x}_sun_y_{sun_y}_x_r_{x_radius}_y_r_{y_radius}_sigma_{sigma}.svg')
    plt.show()


maxPhase= 0.457628#gd['26']['FrontPitch']
interact_manual(simulate_gaus, det_idx=widgets.IntSlider(min=0,max=32,value=19,step=1),
         sun_x=widgets.IntSlider(min=-3600,max=3600,value=0, step=1),
         sun_y=widgets.IntSlider(min=-3600,max=3600,value=0,step=1),
         x_radius=widgets.IntSlider(min=0,max=340,value=30, step=1),
         y_radius=widgets.IntSlider(min=0,max=340,value=30,step=1),
            sigma=widgets.IntSlider(min=0,max=200,value=10,step=1),
                 steps=widgets.IntSlider(min=10,max=100,value=20,step=1)
        
         
        )


interactive(children=(IntSlider(value=19, description='det_idx', max=32), IntSlider(value=0, description='sun_…

<function __main__.simulate_gaus(det_idx, sun_x, sun_y, x_radius=30, y_radius=30, sigma=10, rho=0, steps=40)>